In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hybrid-result-1/hybrid_results (1).json
/kaggle/input/public-test-for-corpus/public_test.json
/kaggle/input/legal-corpus/legal_corpus.json


In [2]:
import json

# Proper way to load standard JSON
with open("/kaggle/input/legal-corpus/legal_corpus.json", "r", encoding="utf-8") as f:
    corpus_data = json.load(f)  # ✅ returns list of dicts

with open("/kaggle/input/public-test-for-corpus/public_test.json", "r", encoding="utf-8") as f:
    query_data = json.load(f)
with open("/kaggle/input/hybrid-result-1/hybrid_results (1).json", "r", encoding="utf-8") as f:
    query_laws = json.load(f)

In [3]:
corpus_articles = []
for doc in corpus_data:
    for article in doc["content"]:
        corpus_articles.append({
            "aid": article["aid"],
            "content": article["content_Article"]
        })

corpus_articles[0]

{'aid': 0,
 'content': '1. Thông tư này quy định mã số, tiêu chuẩn chuyên môn, nghiệp vụ và xếp lương đối với các ngạch công chức chuyên ngành Ngân hàng.\n\n2. Thông tư này áp dụng đối với công chức làm việc tại các đơn vị thuộc Ngân hàng Nhà nước Việt Nam (gọi tắt là Ngân hàng Nhà nước).'}

In [4]:
qid_to_laws = {item["qid"]: item["relevant_laws"] for item in query_laws}

# Update query_data
for query in query_data:
    if query["id"] in qid_to_laws:
        query["relevant_laws"] = qid_to_laws[query["id"]]


In [5]:
from sentence_transformers import CrossEncoder
import json

# 1. Load the Cross-Encoder model
model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

2025-08-07 04:22:55.227117: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754540575.599126      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754540575.716434      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [6]:
reranked_results = []
aid_to_content = {doc["aid"]: doc["content"] for doc in corpus_articles}

for query in query_data:
    qid = query["id"]
    question = query["question"]
    candidate_ids = query["relevant_laws"]
    MAX_CONTENT_CHARS = 2000  # or 2000, adjust as needed

# Build (question, article) pairs with truncation
    pairs = [
        (question, aid_to_content[aid][:MAX_CONTENT_CHARS])
        for aid in candidate_ids if aid in aid_to_content
    ]
    # Build (query, article) pairs

    # Score them
    scores = model.predict(pairs)

    # Sort by score
    reranked = sorted(zip(candidate_ids, scores), key=lambda x: x[1], reverse=True)
    reranked_ids = [aid for aid, _ in reranked]

    reranked_results.append({
        "id": qid,
        "question": question,
        "reranked_laws": reranked_ids
    })

# 5. Save to JSON file


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
import json

# Extract top 3 relevant_laws and save with "qid"
output_data = []
for item in reranked_results:
    output_data.append({
        "qid": item["id"],
        "relevant_laws": item["reranked_laws"][:3]  # top 3 only
    })

# Save to JSON
with open("results.json", "w", encoding="utf-8") as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print("✅ Saved top 3 reranked laws per query to reranked_results.json")


✅ Saved top 3 reranked laws per query to reranked_results.json
